In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
feature_name_df = pd.read_csv("/content/drive/MyDrive/git_colab/human_act/features.txt", sep ='\s+',
                              header = None, names = ['column_index','column_name'])
feature_name = feature_name_df.iloc[:,1].values.tolist()
feature_name[:10]

feature_dup_df = feature_name_df.groupby('column_name').count()
print(feature_dup_df[feature_dup_df['column_index'] > 1].count())
feature_dup_df[feature_dup_df['column_index'] > 1]

column_index    42
dtype: int64


,column_index
column_name,
"fBodyAcc-bandsEnergy()-1,16",3
"fBodyAcc-bandsEnergy()-1,24",3
"fBodyAcc-bandsEnergy()-1,8",3
"fBodyAcc-bandsEnergy()-17,24",3
"fBodyAcc-bandsEnergy()-17,32",3
"fBodyAcc-bandsEnergy()-25,32",3
"fBodyAcc-bandsEnergy()-25,48",3
"fBodyAcc-bandsEnergy()-33,40",3
"fBodyAcc-bandsEnergy()-33,48",3


In [ ]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index() 
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                           if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [ ]:
import pandas as pd

def get_human_dataset( ):
    feature_name_df = pd.read_csv("/content/drive/MyDrive/git_colab/human_act/features.txt", sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    X_train = pd.read_csv('/content/drive/MyDrive/git_colab/human_act/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('/content/drive/MyDrive/git_colab/human_act/test/X_test.txt',sep='\s+', names=feature_name)
    
    y_train = pd.read_csv('/content/drive/MyDrive/git_colab/human_act/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('/content/drive/MyDrive/git_colab/human_act/test/y_test.txt',sep='\s+',header=None,names=['action'])
     
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state = 1)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(accuracy)
print(dt_clf.get_params())

0.8642687478791992
{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 1, 'splitter': 'best'}


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [8,12,16,24],
    'min_samples_split' : [16,24]
}

grid_cv = GridSearchCV(dt_clf, param_grid = params, scoring = 'accuracy', cv = 5, verbose = 1)
grid_cv.fit(X_train, y_train)
print(grid_cv.best_score_)
print(grid_cv.best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
0.8457647858599593
{'max_depth': 8, 'min_samples_split': 16}


In [ ]:
from sklearn.ensemble import RandomForestClassifier
params = {
    'max_depth' : [8,16,24],
    'min_samples_leaf' : [1,6,12],
    'min_samples_split' : [2,6,10]
}
rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv=2, n_jobs = -1)
grid_cv.fit(X_train, y_train)
print(grid_cv.best_params_)
print(grid_cv.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'max_depth': 16, 'min_samples_leaf': 6, 'min_samples_split': 2}
0.9164853101196953


# 파이썬 XGBoost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset = load_breast_cancer()
features = dataset.data
labels = dataset.target

cancer_df = pd.DataFrame(data = features, columns = dataset.feature_names)
cancer_df['target'] = labels
cancer_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [ ]:
X_features = cancer_df.iloc[:,:-1]
y_label = cancer_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size = 0.2, random_state = 9)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 150)
print(X_train.shape, X_test.shape)
print(X_tr.shape, X_val.shape)

(455, 30) (114, 30)
(409, 30) (46, 30)


In [ ]:
dtr = xgb.DMatrix(data = X_tr, label = y_tr)
dval = xgb.DMatrix(data = X_val, label = y_val)
dtest = xgb.DMatrix(data = X_test, label = y_test)

In [ ]:
params = {
    'max_depth' : 3,
    'eta' : 0.05,
    'objective' : 'binary:logistic',
    'eval_metric' : 'logloss'
}
num_rounds = 400

In [ ]:
eval_list = [(dtr, 'train'),(dval, 'eval')]
xgb_model = xgb.train(params = params, dtrain = dtr, num_boost_round=num_rounds, 
                    early_stopping_rounds=50, evals = eval_list)

In [ ]:
pred_probs = xgb_model.predict(dtest)
preds = [1 if x >0.5 else 0 for x in pred_probs]
preds[:10]

[1, 1, 0, 1, 1, 0, 0, 0, 1, 0]

# 사이킷런 XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimators = 400, learning_rate = 0.05, max_depth = 3, eval_metirc = 'logloss')
xgb_wrapper.fit(X_train, y_train, verbose= True)
w_preds = xgb_wrapper.predict(X_test)
x_pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

In [ ]:
x_pred_proba[:10]

array([8.2638496e-01, 9.9306983e-01, 1.3562391e-03, 9.3454880e-01,
       9.9860942e-01, 9.1456407e-04, 3.9328716e-04, 5.2139617e-04,
       9.9806803e-01, 4.0116769e-04], dtype=float32)

In [ ]:
xgb_wrapper = XGBClassifier(n_estimators = 500, learning_rate = 0.05, max_depth = 3)
evals = [(X_tr, y_tr), (X_val, y_val)]
xgb_wrapper.fit(X_train, y_train, early_stopping_rounds = 50, eval_metric = 'logloss',
                eval_set = evals, verbose= True)
w_preds = xgb_wrapper.predict(X_test)

# LGBM

In [ ]:
import lightgbm
print(lightgbm.__version__)

2.2.3


In [ ]:
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

dataset = load_breast_cancer()
labels = dataset.target
cancer_df = pd.DataFrame(data = dataset.data, columns = dataset.feature_names)
cancer_df['target'] = labels
cancer_df

X_features = cancer_df.iloc[:,:-1]
y_label = cancer_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size = 0.2, random_state = 9)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 150)

lgb_wrapper = LGBMClassifier(n_estimators = 500, learning_rate = 0.05)
evals = [(X_tr, y_tr), (X_val, y_val)]
lgb_wrapper.fit(X_tr, y_tr, early_stopping_rounds = 50, eval_metric = 'logloss',
                eval_set = evals, verbose= True)

preds = lgb_wrapper.predict(X_test)

[1]	training's binary_logloss: 0.625086	training's binary_logloss: 0.625086	valid_1's binary_logloss: 0.609271	valid_1's binary_logloss: 0.609271
Training until validation scores don't improve for 50 rounds.
[2]	training's binary_logloss: 0.588358	training's binary_logloss: 0.588358	valid_1's binary_logloss: 0.571836	valid_1's binary_logloss: 0.571836
[3]	training's binary_logloss: 0.555492	training's binary_logloss: 0.555492	valid_1's binary_logloss: 0.537838	valid_1's binary_logloss: 0.537838
[4]	training's binary_logloss: 0.526354	training's binary_logloss: 0.526354	valid_1's binary_logloss: 0.509525	valid_1's binary_logloss: 0.509525
[5]	training's binary_logloss: 0.498958	training's binary_logloss: 0.498958	valid_1's binary_logloss: 0.482594	valid_1's binary_logloss: 0.482594
[6]	training's binary_logloss: 0.473367	training's binary_logloss: 0.473367	valid_1's binary_logloss: 0.456565	valid_1's binary_logloss: 0.456565
[7]	training's binary_logloss: 0.449905	training's binary_logl

# Hyperopt

In [ ]:
from hyperopt import fmin, tpe, Trials
import numpy as np
from hyperopt import STATUS_OK
from hyperopt import hp

In [ ]:
search_space = {'x' : hp.quniform('x', -10, 10, 1), 'y' : hp.quniform('y', -15, 15, 1)}
search_space

def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y
    return retval 

trial_val = Trials()

best_01 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=20
               , trials=trial_val,
              )
print('best:', best_01)

100%|██████████| 20/20 [00:00<00:00, 457.07it/s, best loss: -144.0]
best: {'x': 6.0, 'y': 9.0}


In [ ]:
trial_val.results

[{'loss': 64.0, 'status': 'ok'},
 {'loss': 161.0, 'status': 'ok'},
 {'loss': -144.0, 'status': 'ok'},
 {'loss': 236.0, 'status': 'ok'},
 {'loss': 284.0, 'status': 'ok'},
 {'loss': 165.0, 'status': 'ok'},
 {'loss': 184.0, 'status': 'ok'},
 {'loss': -59.0, 'status': 'ok'},
 {'loss': -39.0, 'status': 'ok'},
 {'loss': 281.0, 'status': 'ok'},
 {'loss': 149.0, 'status': 'ok'},
 {'loss': -59.0, 'status': 'ok'},
 {'loss': 124.0, 'status': 'ok'},
 {'loss': -135.0, 'status': 'ok'},
 {'loss': 136.0, 'status': 'ok'},
 {'loss': 176.0, 'status': 'ok'},
 {'loss': 125.0, 'status': 'ok'},
 {'loss': 144.0, 'status': 'ok'},
 {'loss': 229.0, 'status': 'ok'},
 {'loss': 89.0, 'status': 'ok'}]

In [ ]:
trial_val.vals

{'x': [-2.0,
  1.0,
  6.0,
  4.0,
  8.0,
  -5.0,
  -2.0,
  1.0,
  -9.0,
  -9.0,
  -3.0,
  1.0,
  8.0,
  -5.0,
  4.0,
  -4.0,
  5.0,
  8.0,
  -3.0,
  -3.0],
 'y': [-3.0,
  -8.0,
  9.0,
  -11.0,
  -11.0,
  -7.0,
  -9.0,
  3.0,
  6.0,
  -10.0,
  -7.0,
  3.0,
  -3.0,
  8.0,
  -6.0,
  -8.0,
  -5.0,
  -4.0,
  -11.0,
  -4.0]}

# HyperOpt + XGBoost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import warnings
warnings.filterwarnings('ignore')

dataset = load_breast_cancer()
features = dataset.data
labels = dataset.target

cancer_df = pd.DataFrame(data = features, columns = dataset.feature_names)
cancer_df['target'] = labels
cancer_df

X_features = cancer_df.iloc[:,:-1]
y_label = cancer_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size = 0.2, random_state = 9)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 150)
print(X_train.shape, X_test.shape)
print(X_tr.shape, X_val.shape)

(455, 30) (114, 30)
(409, 30) (46, 30)


In [ ]:
from hyperopt import hp
xgb_search_space = {'max_depth' : hp.quniform('max_depth',5,20,1),
                    'min_child_weight' : hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate' :hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1) }

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators = 500, max_depth = int(search_space['max_depth']),
                            min_child_weight = int(search_space['min_child_weight']),
                            learning_rate = search_space['learning_rate'],
                            colsample_bytree = search_space['colsample_bytree'],
                            eveal_metric = 'logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring = 'accuracy', cv = 3)
    return {'loss' : -1 * np.mean(accuracy), 'status':STATUS_OK}

In [ ]:
from hyperopt import fmin, tpe, Trials

tral_val = Trials()
best = fmin(fn=objective_func,
            space = xgb_search_space,
            algo = tpe.suggest,
            max_evals=50,
            trials = tral_val)
print('best:', best)

100%|██████████| 50/50 [00:15<00:00,  3.14it/s, best loss: -0.9669890786569072]
best: {'colsample_bytree': 0.5009751008832034, 'learning_rate': 0.1590193796651615, 'max_depth': 6.0, 'min_child_weight': 1.0}


# 산탄데르 은행 고객 예측

In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/git_colab/dataset/train.csv')

In [17]:
# 0이 만족한 고객
# 1이 불만족한 고객
print(df['TARGET'].value_counts())
print(df['TARGET'].value_counts()[1]/df['TARGET'].value_counts().sum())

0    73012
1     3008
Name: TARGET, dtype: int64
0.0395685345961589


In [25]:
df['var3'].value_counts()[df['var3'].value_counts().index < 0]

-999999    116
Name: var3, dtype: int64

In [26]:
df['var3'].replace(-999999, 2, inplace = True)
df['var3'].drop(columns = 'ID', axis = 1, inplace = True)

In [30]:
X_features = df.iloc[:,:-1]
y_labels = df.iloc[:,-1]

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size = 0.2, random_state = 0)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state = 0)

In [34]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimators = 500, learning_rate = 0.05, random_state = 156)
xgb_clf.fit(X_tr,y_tr, early_stopping_rounds = 100, eval_metric = 'auc', eval_set = [(X_tr, y_tr),(X_val, y_val)])
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.803388	validation_1-auc:0.797852
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.812619	validation_1-auc:0.796774
[2]	validation_0-auc:0.811648	validation_1-auc:0.798697
[3]	validation_0-auc:0.812568	validation_1-auc:0.797626
[4]	validation_0-auc:0.814884	validation_1-auc:0.798837
[5]	validation_0-auc:0.814676	validation_1-auc:0.80085
[6]	validation_0-auc:0.814879	validation_1-auc:0.799201
[7]	validation_0-auc:0.815144	validation_1-auc:0.798952
[8]	validation_0-auc:0.81629	validation_1-auc:0.801921
[9]	validation_0-auc:0.816236	validation_1-auc:0.800339
[10]	validation_0-auc:0.815842	validation_1-auc:0.80191
[11]	validation_0-auc:0.816102	validation_1-auc:0.800052
[12]	validation_0-auc:0.818573	validation_1-auc:0.805199
[13]	validation_0-auc:0.816937	validation_1-auc:0.801944
[14]	validation_0-auc:0.816405	validation_1-auc:0.801992


In [38]:
from hyperopt import hp
import numpy as np
# max_depth는 5에서 15까지 1간격으로, min_child_weight는 1에서 6까지 1간격으로
# colsample_bytree는 0.5에서 0.95사이, learning_rate는 0.01에서 0.2사이 정규 분포된 값으로 검색. 

xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1), 
                    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2)
}

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth'])
                           , min_child_weight=int(search_space['min_child_weight'])
                            , colsample_bytree=search_space['colsample_bytree']
                            , learning_rate=search_space['learning_rate']
                           )
    roc_auc_list= []
    
    kf = KFold(n_splits=3)
    for tr_index, val_index in kf.split(X_train):
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric='auc'
                   , eval_set=[(X_tr, y_tr), (X_val, y_val)])
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:, 1])
        roc_auc_list.append(score)
    
    return -1 * np.mean(roc_auc_list)

from hyperopt import fmin, tpe, Trials

trials = Trials()

best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trials)

print('best:', best)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

[35]	validation_0-auc:0.874961	validation_1-auc:0.831862

[36]	validation_0-auc:0.874339	validation_1-auc:0.830461

[37]	validation_0-auc:0.874889	validation_1-auc:0.831413

[38]	validation_0-auc:0.875677	validation_1-auc:0.831276

[39]	validation_0-auc:0.875815	validation_1-auc:0.830224

[40]	validation_0-auc:0.877112	validation_1-auc:0.831275

[41]	validation_0-auc:0.878746	validation_1-auc:0.832224

[42]	validation_0-auc:0.880287	validation_1-auc:0.832833

[43]	validation_0-auc:0.880955	validation_1-auc:0.83254

[44]	validation_0-auc:0.88247	validation_1-auc:0.833106

[45]	validation_0-auc:0.882268	validation_1-auc:0.832339

[46]	validation_0-auc:0.883365	validation_1-auc:0.833429

[47]	validation_0-auc:0.883286	validation_1-auc:0.833253

[48]	validation_0-auc:0.884737	validation_1-auc:0.834298

[49]	validation_0-auc:0.885334	validation_1-auc:0.833988

[50]	validation_0-auc:0.885579	validation_1-auc:0.833537

[51]	validation_0-auc:0.885554	valida

# 베이지안 최적화를 통해 파라미터 튜닝을 진행하는데 있어, XGBoost와 LGBM을 사용해봤는데 LGBM이 훨씬 빠르다는 걸 알았다.